# REST APIs & KI Agents Demo 🧠💻
Dieses Notebook demonstriert:
1. Verwendung einfacher REST APIs in Python
2. Nutzung von KI-Agents (ChatGPT) zur Analyse der API-Daten
3. Kombination von API-Daten und KI-Agenten

## 🚀 Schritt 1: Installation notwendiger Pakete
Falls noch nicht installiert, bitte ausführen:

In [1]:
%pip install requests openai pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Imports
import requests
import pandas as pd

## Abschnitt 1: Verwendung einfacher REST APIs

In [9]:
# Beispiel-API: JSONPlaceholder (Fake REST API für Demo)
url = "https://jsonplaceholder.typicode.com/posts"
# url = "https://data-api.ecb.europa.eu/service/data/EXR/D.USD.EUR.SP00.A?format=jsondata&lastNObservations=14"

# GET-Anfrage an die API
response = requests.get(url)

# Prüfen, ob die Anfrage erfolgreich war
if response.status_code == 200:
    data = response.json()
    print("Erste 3 Posts von JSONPlaceholder:")
    print(data[:3])
else:
    print(f"Fehler bei der API-Anfrage: {response.status_code}")

# Umwandeln in DataFrame für bessere Übersicht
df = pd.DataFrame(data)
df.head(5)

Erste 3 Posts von JSONPlaceholder:
[{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}, {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'}, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}]


,userId,id,title,body
0,1,1,sunt aut facere repellat provident occaecati e...,quia et suscipit\nsuscipit recusandae consequu...
1,1,2,qui est esse,est rerum tempore vitae\nsequi sint nihil repr...
2,1,3,ea molestias quasi exercitationem repellat qui...,et iusto sed quo iure\nvoluptatem occaecati om...
3,1,4,eum et est occaecati,ullam et saepe reiciendis voluptatem adipisci\...
4,1,5,nesciunt quas odio,repudiandae veniam quaerat sunt sed\nalias aut...


In [4]:
# Optional: Filtern der Posts eines bestimmten Benutzers
user_posts = df[df['userId'] == 1]
print("\nPosts des Benutzers 1:")
print(user_posts[['id','title']])


Posts des Benutzers 1:
   id                                              title
0   1  sunt aut facere repellat provident occaecati e...
1   2                                       qui est esse
2   3  ea molestias quasi exercitationem repellat qui...
3   4                               eum et est occaecati
4   5                                 nesciunt quas odio
5   6                 dolorem eum magni eos aperiam quia
6   7                               magnam facilis autem
7   8                           dolorem dolore est ipsam
8   9  nesciunt iure omnis dolorem tempora et accusan...
9  10                        optio molestias id quia eum


## Abschnitt 2: Nutzung eines KI-Agents (ChatGPT & Ollama)

### ChatGPT
Für chatGPT ist ein API-Key notwendig (paid service 💰). Dieser API-Key wird in einer .env Datei abgelegt (Sicherheit!) und wie ein Passwort nicht geteilt.

In [12]:
import openai
from dotenv import load_dotenv
import os

# Hier wird die .env Datei geladen
load_dotenv()
openai.api_key = os.getenv("OPEN_API_KEY")

def chatgpt_query(prompt):
    """
    Funktion, um eine Anfrage an ChatGPT zu stellen
    """
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

# Beispiel-Anfrage
prompt = "Fasse die Inhalte der ersten 5 Posts zusammen."
summary = chatgpt_query(prompt)
print("\nChatGPT-Zusammenfassung:")
print(summary)


ChatGPT-Zusammenfassung:
Als KI-Assistent brauche ich konkrete Informationen, auf die ich zugreifen kann. Im Moment habe ich keine Posts, die ich für Sie zusammenfassen könnte. Bitte stellen Sie sicher, dass Sie mir genügend Details oder Kontext geben, damit ich Ihnen helfen kann.


 ### Ollama
Ollama kann lokal oder remote Modelle nutzen. Hier ein Beispiel mit Python:
  -  Installiere vorher `ollama` Python-Package: `pip install ollama`
  - Stelle sicher, dass Ollama-Daemon läuft, z.B. `ollama serve`"

In [14]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Beispiel mit Ollama
try:
    from ollama import Ollama

    ollama_client = Ollama()

    def ollama_query(prompt, model='llama2'):
        """Frage an Ollama-Modell"""
        response = ollama_client.chat(model=model, prompt=prompt)
        return response['text']

    # Beispiel-Anfrage
    ollama_prompt = "Fasse die Inhalte der ersten 5 Posts zusammen."
    ollama_summary = ollama_query(ollama_prompt)
    print("\nOllama-Zusammenfassung:")
    print(ollama_summary)

except ImportError:
    print("Ollama-Package nicht installiert. Bitte 'pip install ollama' ausführen.")

Ollama-Package nicht installiert. Bitte 'pip install ollama' ausführen.


## Abschnitt 3: Verbindung von API + KI

In [17]:
# Wir holen Post-Titel und lassen ChatGPT ein gemeinsames Thema finden
post_titles = df['title'].head(5).tolist()
analysis_prompt = f"Analysiere die folgenden Post-Titel und finde ein gemeinsames Thema:\n{post_titles}"
theme_analysis = chatgpt_query(analysis_prompt)
print("\nGemeinsames Thema der Posts laut ChatGPT:")
print(theme_analysis)


Gemeinsames Thema der Posts laut ChatGPT:
Die Post-Titel sind leider sehr allgemein und nicht klar genug, um ein bestimmtes gemeinsames Thema zu identifizieren. Sie scheinen zufällige Phrasen oder Sätze zu sein, möglicherweise aus einer lateinischen oder italienischen Sprache. Ohne zusätzlichen Kontext oder spezifischere Begriffe ist es schwierig, ein gemeinsames Thema zu bestimmen.


## Abschnitt 4 (Optional): Interaktion mit Copilot/Gemini
Hinweis: GitHub Copilot arbeitet IDE-basiert, du kannst aber KI-generierte Code-Snippets direkt hier einfügen.
Google Gemini oder andere KI-Agenten funktionieren ähnlich wie ChatGPT über REST-APIs.